<a href="https://colab.research.google.com/github/kevinle350/ML_Reddit_Stock_Sentiment_Predictor/blob/main/DLProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt
plt.style.use('ggplot')
from sklearn.metrics import roc_curve, auc
from sklearn.metrics import classification_report, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from sklearn.model_selection import train_test_split
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.cm as cm
from matplotlib import rcParams
from collections import Counter
from nltk.tokenize import RegexpTokenizer
import re
import string
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
%matplotlib inline
 
import warnings
warnings.filterwarnings("ignore")

from google.colab import drive 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!mkdir -p drive
!google-drive-ocamlfuse drive
import os 
folder = os.path.join('/content/drive/MyDrive/UIUC/NOBE/training.1600000.processed.noemoticon.csv')


/bin/bash: google-drive-ocamlfuse: command not found


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/UIUC/NOBE/training.1600000.processed.noemoticon.csv", encoding = "ISO-8859-1", engine="python")


In [ ]:
df.columns = ["label", "time", "date", "query", "username", "text"]
df.head()

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
df.head()

,label,time,date,query,username,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
df['label'][df['label'] == 0] = -1
df['label'][df['label'] == 2] = 0
df['label'][df['label'] == 4] = 1
df['label']

0         -1
1         -1
2         -1
3         -1
4         -1
          ..
1599994    1
1599995    1
1599996    1
1599997    1
1599998    1
Name: label, Length: 1599999, dtype: int64

In [ ]:
df['text'] = df['text'].str.lower()
df['text']

0          is upset that he can't update his facebook by ...
1          @kenichan i dived many times for the ball. man...
2            my whole body feels itchy and like its on fire 
3          @nationwideclass no, it's not behaving at all....
4                              @kwesidei not the whole crew 
                                 ...                        
1599994    just woke up. having no school is the best fee...
1599995    thewdb.com - very cool to hear old walt interv...
1599996    are you ready for your mojo makeover? ask me f...
1599997    happy 38th birthday to my boo of alll time!!! ...
1599998    happy #charitytuesday @thenspcc @sparkscharity...
Name: text, Length: 1599999, dtype: object

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stop_words = stopwords.words('english')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:
def clean_stop_words(text):
  word_list = str(text).split()
  cleaned_word_list = []
  for word in word_list:
    if word not in stop_words:
      cleaned_word_list.append(word)
  return " ".join(cleaned_word_list)

In [ ]:
def clean_at(text):
  split_text = text.split()
  non_at_words = []
  for word in split_text:
    if not re.search(".*@.*", word):
      non_at_words.append(word)
  return " ".join(non_at_words)

In [ ]:
english_punctuations = string.punctuation
punctuations_list = english_punctuations
def clean_punctuation(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

In [ ]:
def clean_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [ ]:
def clean_urls(data):
    return re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',data)

In [ ]:
def clean_nums(data):
    return re.sub('[0-9]+', '', data)

In [ ]:
print("Cleaning stop words...")
df['text'] = df['text'].apply(lambda tweet: clean_stop_words(tweet))
print("Cleaning words with @...")
df['text'] = df['text'].apply(lambda tweet: clean_at(tweet))
print("Cleaning punctuation...")
df['text'] = df['text'].apply(lambda tweet: clean_punctuation(tweet))
print("Cleaning urls...")
df['text'] = df['text'].apply(lambda tweet: clean_urls(tweet))
print("Cleaning repeating characters...")
df['text'] = df['text'].apply(lambda tweet: clean_repeating_char(tweet))
print("Cleaning numeric characters...")
df['text'] = df['text'].apply(lambda tweet: clean_nums(tweet))
print("Done cleaning")
print(df.head())


Cleaning stop words...
Cleaning words with @...
Cleaning punctuation...
Cleaning urls...
Cleaning repeating characters...
Cleaning numeric characters...
Done cleaning
   label  ...                                               text
0     -1  ...  upset cant update facebok texting it might cry...
1     -1  ...  dived many times bal managed save  rest go bounds
2     -1  ...                    whole body fels itchy like fire
3     -1  ...           no behaving al im mad here cant se there
4     -1  ...                                         whole crew

[5 rows x 6 columns]


In [ ]:
tokenizer = RegexpTokenizer(r'\w+')
df['text'] = df['text'].apply(tokenizer.tokenize)

In [ ]:
df.head()
df['text']

0          [upset, cant, update, facebok, texting, it, mi...
1          [dived, many, times, bal, managed, save, rest,...
2                     [whole, body, fels, itchy, like, fire]
3          [no, behaving, al, im, mad, here, cant, se, th...
4                                              [whole, crew]
                                 ...                        
1599994                [woke, up, schol, best, feling, ever]
1599995    [thewdbcom, col, hear, old, walt, interviews, ...
1599996                [ready, mojo, makeover, ask, details]
1599997    [hapy, th, birthday, bo, al, time, tupac, amar...
1599998                               [hapy, charitytuesday]
Name: text, Length: 1599999, dtype: object

In [ ]:
ps = PorterStemmer()
def stemming_test(data):
  text = [ps.stem(word) for word in data]
  return data
  df['text'] = df['text'].apply(lambda x: stemming_test(x))

In [ ]:
nltk.download('wordnet')
lemmatizer = nltk.WordNetLemmatizer()
def lemmatizing_test(data):
  text = [lemmatizer.lemmatize(word) for word in data]
  return data

df['text'] = df['text'].apply(lambda x: lemmatizing_test(x))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
max_len = 500
tok = Tokenizer(num_words=2000)
tok.fit_on_texts(df['text'])
sequences = tok.texts_to_sequences(df['text'])
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [ ]:
df['label'][df['label'] == 0] = 0.5
df['label'][df['label'] == -1] = 0
df['label'][df['label'] == 1] = 1

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(sequences_matrix, df['label'] , test_size = 0.7, random_state=42)

In [ ]:
def nm_model():
  model = Input(shape = [500]);
  layer1 = model
  model = Embedding(input_dim = 2000, output_dim = 500, input_length = 500)(model);
  lstm = LSTM(64);
  model = lstm(model)
  model = Dense(256)(model)
  model = Activation('relu')(model)
  model = Dropout(.5)(model)
  model = Dense(1)(model)
  b = Activation("sigmoid")
  model = b(model)
  return Model(inputs = layer1, outputs = model)

In [ ]:
model = nm_model()
model.compile(loss='binary_crossentropy', optimizer=RMSprop(), metrics=['accuracy'])
history=model.fit(X_train[:40000], Y_train[:40000], batch_size=100, epochs=6, validation_split=0.1)
print('Training finished')

Epoch 1/6
360/360 [==============================] - 62s 80ms/step - loss: 0.5887 - accuracy: 0.6733 - val_loss: 0.5033 - val_accuracy: 0.7558
Epoch 2/6
360/360 [==============================] - 28s 78ms/step - loss: 0.4833 - accuracy: 0.7656 - val_loss: 0.4908 - val_accuracy: 0.7613
Epoch 3/6
360/360 [==============================] - 28s 79ms/step - loss: 0.4615 - accuracy: 0.7789 - val_loss: 0.4962 - val_accuracy: 0.7573
Epoch 4/6
360/360 [==============================] - 29s 79ms/step - loss: 0.4386 - accuracy: 0.7933 - val_loss: 0.4969 - val_accuracy: 0.7590
Epoch 5/6
360/360 [==============================] - 29s 80ms/step - loss: 0.4261 - accuracy: 0.7994 - val_loss: 0.5121 - val_accuracy: 0.7492
Epoch 6/6
360/360 [==============================] - 29s 79ms/step - loss: 0.4014 - accuracy: 0.8123 - val_loss: 0.5154 - val_accuracy: 0.7533
Training finished


In [ ]:
accr1 = model.evaluate(X_test, Y_test)

35000/35000 [==============================] - 561s 16ms/step - loss: 0.5382 - accuracy: 0.7419
